In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-05-11 04:56:37--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  77.8MB/s    in 0.3s    

2025-05-11 04:56:38 (77.8 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [7]:
# add your code here - consider creating a new cell for each section of code
user_counts = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

# Filter books with at least 100 ratings
book_counts = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)]

# Create user-item matrix
book_user_mat = df_ratings.pivot(index='isbn', columns='user', values='rating').fillna(0)
book_user_sparse = csr_matrix(book_user_mat.values)

# Fit Nearest Neighbors model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_user_sparse)

# Build title <-> isbn maps
isbn_to_title = df_books.set_index('isbn')['title'].drop_duplicates()
title_to_isbn = df_books.drop_duplicates(subset='title').set_index('title')['isbn']

# Build index maps
isbn_to_index = {isbn: idx for idx, isbn in enumerate(book_user_mat.index)}
index_to_isbn = {idx: isbn for isbn, idx in isbn_to_index.items()}

In [5]:
# function to return recommended books - this will be tested
def get_recommends(book=""):
    if book not in title_to_isbn:
        return [book, []]

    isbn = title_to_isbn[book]
    if isbn not in isbn_to_index:
        return [book, []]

    book_idx = isbn_to_index[isbn]
    distances, indices = model.kneighbors(book_user_mat.iloc[book_idx, :].values.reshape(1, -1), n_neighbors=6)

    recommends = []
    for i in range(1, len(indices[0])):  # Skip the input book itself
        similar_idx = indices[0][i]
        similar_isbn = book_user_mat.index[similar_idx]
        similar_title = isbn_to_title.get(similar_isbn, "Unknown Title")
        recommends.append([similar_title, float(distances[0][i])])

    return [book, recommends]

In [6]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', np.float32(0.7234864)], ["The Pilot's Wife : A Novel", np.float32(0.81926787)], ['The Joy Luck Club', np.float32(0.81986046)], ['The Notebook', np.float32(0.82366824)], ['Bel Canto: A Novel', np.float32(0.8247875)]]]
You haven't passed yet. Keep trying!
